In [1]:
import shots_data_retriever
from shots_data_retriever import ShotsDataRetriever
import importlib
import warnings
import pandas as pd

# Filter dtype warnings. Column 10 can contain str, int, and float types 
# which spams a warning in the output
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
importlib.reload(shots_data_retriever)

shotsDataRetriever = ShotsDataRetriever()

In [10]:
# Average shot rate
def get_league_avg_shot_rate_by_coordinate(year: str):
    df = shotsDataRetriever.get_season_shots(year)

    single_game_time = 1 # in hours, assumption from google doc
    total_games = df['game_id'].nunique()
    total_game_time = single_game_time * total_games

    shot_location = df.groupby(['x_coord', 'y_coord']).size().reset_index(name='shot_count')
    shot_location['shot_rate'] = shot_location['shot_count'] / total_game_time

    return shot_location

In [11]:
# Shot rate by team
def get_team_avg_shot_rate_by_coordinate(year: str, team_id: int):
    df = shotsDataRetriever.get_season_shots_for_team(year, team_id)

    single_game_time = 1 # in hours, assumption from google doc
    total_games = df['game_id'].nunique()
    total_game_time = single_game_time * total_games

    shot_location = df.groupby(['x_coord', 'y_coord']).size().reset_index(name='shot_count')
    shot_location['shot_rate'] = shot_location['shot_count'] / total_game_time

    return shot_location

In [39]:
def get_team_excess_shot_rate(year: str, team_id: int):
    team_df = get_team_avg_shot_rate_by_coordinate(year, team_id)
    league_df = get_league_avg_shot_rate_by_coordinate(year)

    result_df = team_df
    result_df['team_shot_rate'] = result_df['shot_rate'].copy()
    result_df = result_df.drop(columns=['shot_rate'])
    result_df['league_shot_rate'] = league_df['shot_rate']
    result_df['excess_shot_rate'] = team_df['shot_rate'] - league_df['shot_rate']
    
    return result_df

df = get_team_excess_shot_rate('2017', 16)

# Validate the values - about half of the values should be negative
print(f"positive: {df[df['excess_shot_rate'] >= 0]['excess_shot_rate'].count()}")
print(f"negative: {df[df['excess_shot_rate'] < 0]['excess_shot_rate'].count()}")

print(df)

positive: 1895
negative: 0
      x_coord  y_coord  shot_count  team_shot_rate  league_shot_rate  \
0         0.0    -32.0           1        0.012195          0.000738   
1         1.0    -18.0           1        0.012195          0.001476   
2         2.0     29.0           1        0.012195          0.001476   
3         3.0     -9.0           1        0.012195          0.000738   
4         3.0     16.0           1        0.012195          0.000738   
...       ...      ...         ...             ...               ...   
1890     93.0     18.0           1        0.012195          0.006642   
1891     95.0     10.0           1        0.012195          0.003690   
1892     95.0     11.0           1        0.012195          0.005904   
1893     99.0     -4.0           1        0.012195          0.005166   
1894     99.0     10.0           1        0.012195          0.005904   

      excess_shot_rate  
0             0.011457  
1             0.010719  
2             0.010719  
3       

In [5]:
# Difference from league average
full_df = pd.DataFrame()
temp_df = pd.DataFrame()
temp_df['team_id'] = shot_location_group['team_id']
temp_df['x_coord'] = shot_location_group['x_coord']
temp_df['y_coord'] = shot_location_group['y_coord']

temp_df = temp_df.groupby('team_id')
for team_group in temp_df:
    team_id = team_group[1]['team_id'].iloc[0]
    x_coord = team_group[1]['x_coord'].iloc[0]
    y_coord = team_group[1]['y_coord'].iloc[0]
    merge_df = pd.merge(
        shot_location_group,
        shot_location_league[['x_coord', 'y_coord', 'shot_rate']].rename(columns={'shot_rate': 'shot_rate_league'}),
        on=['x_coord', 'y_coord'],
        how='left'
    )
    full_df = pd.concat([full_df, shot_location_group[shot_location_group['team_id'] == team_id]])
    full_df = pd.merge(full_df, merge_df, how='left')
    full_df = full_df.drop('shot_count', axis=1)
    full_df = full_df.drop('time_played', axis=1)
full_df.to_csv('full_df.csv', index=False)

In [32]:
import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
from PIL import Image
from scipy.ndimage import gaussian_filter

In [80]:
def plot_shot_heatmap_plotly(shot_rate_map: pd.DataFrame):
    x = shot_rate_map['x_coord']
    y = shot_rate_map['y_coord']
    shot_rate = shot_rate_map['excess_shot_rate']

    # Create a 2D histogram (heatmap) of the shot rates - obtained using ChatGPT
    heatmap, xedges, yedges = np.histogram2d(
        x, 
        y, 
        bins=[np.linspace(0, 100, 50), 
              np.linspace(-42, 42, 50)], 
        weights=shot_rate)

    heatmap = gaussian_filter(heatmap, sigma=3)

    fig = go.Figure()

    fig.add_layout_image(
        dict(
            source=Image.open("../../figures/nhl_rink.png"),
            x=-100,
            y=42.5,
            xref="x",
            yref="y",
            sizex=200,
            sizey=85,
            opacity=1,
            sizing="stretch",
            xanchor="left",
            yanchor="top",
            layer="below"
        )
    )

    custom_colorscale = [[0, 'rgba(0,0,255,0)'], [0.02, 'rgba(0,0,255,0.05)'], [1, 'rgba(255,0,0,1)']]

    fig.add_trace(go.Heatmap(
        z=heatmap.T,
        x=xedges[:-1],
        y=yedges[:-1],
        colorscale=custom_colorscale,
        colorbar=dict(title='Excess Shot Rate'),
        zmin=0,
        zmax=np.max(heatmap),
        hovertemplate='X: %{x:.2f}<br>Y: %{y:.2f}<br>Excess Shot Rate: %{z:.2f}<extra></extra>',
    ))

    fig.update_layout(
        xaxis=dict(title='X Coordinate (feet)', range=[0, 100]),
        yaxis=dict(title='Y Coordinate (feet)', range=[-42.5, 42.5]),
        title='Shot Distribution on NHL Rink',
        showlegend=False,
        height=425,
        width=580,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    return fig

In [81]:
def generate_shot_rate_plots(year: str):
    teams = shotsDataRetriever.TEAM_IDS[:3]
    print(teams)

    for i, team_id in tqdm(enumerate(teams), desc="Generating Shot Rate Plots", total=len(teams)):
        if i == 0:
            fig = plot_shot_heatmap_plotly(get_team_excess_shot_rate(year, team_id))
            continue

        df = get_team_excess_shot_rate(year, team_id)
        heatmap_fig = plot_shot_heatmap_plotly(df)
        heatmap = heatmap_fig.data[0]
        heatmap.visible = False
        fig.add_trace(heatmap)
    
    fig.update_layout(
        title = f'Excess Shot Rate for Team {team_id} - {year}',
        updatemenus = [
            {
                "buttons": [
                    {
                        "label": f"Team: {team_id}",
                        "method": "update",
                        "args": [
                            {"visible": [i == j for j in range(len(teams))]},  # Show selected trace
                            {"title": f'Excess Shot Rate for Team {team_id} - {year}'},  # Update title
                        ],
                    } for i, team_id in enumerate(teams)
                ],
                "direction": "down",
                "showactive": True,
            }
        ]
    )

    fig.show()

generate_shot_rate_plots('2017')

[1]


Generating Shot Rate Plots: 100%|██████████| 1/1 [00:25<00:00, 25.27s/it]
